In [1]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher


In [2]:
df  = pd.read_pickle("data/data.pkl")
df['Date'] = df['Votation'].map(lambda x : x.split(' ')[0])
df

,Commune,Votation,Electeurs inscrits,Bulletins rentrés,Participation en %,Bulletins valables,Oui,Non,Oui en %,District,Canton,Pays,Date
0,Aeugst am Albis,29.11.1998 Initiative Droleg,1070.0,487.0,45.5,478.0,167.0,311.0,34.9,Affoltern,Zürich,Suisse,29.11.1998
1,Aeugst am Albis,14.06.2015 Initiative sur les bourses d'études,1380.0,706.0,51.2,695.0,186.0,509.0,26.8,Affoltern,Zürich,Suisse,14.06.2015
2,Aeugst am Albis,25.09.2016 Loi fédérale sur le renseignement,1400.0,670.0,47.9,659.0,417.0,242.0,63.3,Affoltern,Zürich,Suisse,25.09.2016
3,Aeugst am Albis,03.03.1991 Encouragement des transports publics,835.0,321.0,38.4,312.0,128.0,184.0,41.0,Affoltern,Zürich,Suisse,03.03.1991
4,Aeugst am Albis,12.02.2017 Réforme de l'imposition des entrepr...,1395.0,759.0,54.4,750.0,318.0,432.0,42.4,Affoltern,Zürich,Suisse,12.02.2017
5,Aeugst am Albis,07.03.2010 Recherche sur l'être humain,1262.0,588.0,46.6,579.0,476.0,103.0,82.2,Affoltern,Zürich,Suisse,07.03.2010
6,Aeugst am Albis,08.02.2004 Initiative pour l'internement des d...,1135.0,590.0,52.0,578.0,266.0,312.0,46.0,Affoltern,Zürich,Suisse,08.02.2004
7,Aeugst am Albis,"22.09.2002 L'or à l'AVS, aux cantons et à la F...",1113.0,610.0,54.8,604.0,325.0,267.0,54.9,Affoltern,Zürich,Suisse,22.09.2002
8,Aeugst am Albis,24.11.2002 Contre les abus dans le droit d'asile,1117.0,662.0,59.3,648.0,282.0,366.0,43.5,Affoltern,Zürich,Suisse,24.11.2002
9,Aeugst am Albis,01.12.1985 Suppression de la vivisection,683.0,363.0,53.1,358.0,135.0,223.0,37.7,Affoltern,Zürich,Suisse,01.12.1985


In [3]:
xls = pd.ExcelFile('data/Recommandations des partis.xls')
nb_sheets = len(xls.sheet_names)
recommand = pd.DataFrame()

for i in range ((2017-1981 + 1)) :
#for i in range (6,7) :
 
    x = xls.parse(i)
    x.columns = range(len(x.columns))
    
    base_nb = x[x.iloc[:,0] == 'Parti 1)'].index[0]


    x=x.dropna(axis = 1 , how = 'all')

    recommand_inter = pd.concat([x.iloc[base_nb:base_nb+1],x.iloc[base_nb+2:base_nb+3],x.dropna()]).transpose()
    recommand_inter.columns = range(len(recommand_inter.columns))
    recommand_inter.iloc[:,0] = recommand_inter.iloc[:,0]+str(2017-i)
    
    recommand_inter.iloc[:,1] = recommand_inter.iloc[:,1].map( lambda x : x[0].upper() + x[1:] ,\
                                                              na_action = 'ignore')
   

    propositions = x[x.iloc[:,0].str.contains('No ').fillna(False)]
    propositions.columns = range (len( propositions.columns))
    
    if  (propositions.iloc[:,1].isnull()).all() :
        if ( 2017-i == 1997) :
            propositions.iloc[:,1] = propositions[0].map ( lambda x : x.split(' ')[1])
            propositions.iloc[:,0] = propositions[0].map ( lambda x : x.split(' ')[0].rstrip())
        else :
                
            propositions.iloc[:,1] = propositions[0].map ( lambda x : x.split(':')[1][1:])
            propositions.iloc[:,0] = propositions[0].map ( lambda x : x.split(':')[0].rstrip())
    
    
    dico_no_propos = propositions.dropna(1).set_index(0).to_dict()
    if ( dico_no_propos.get(1) == None ) :
        
        recommand_inter.iloc[:,1] = recommand_inter.iloc[:,1].map(dico_no_propos)
    else : 
        recommand_inter.iloc[:,1] = recommand_inter.iloc[:,1].map(dico_no_propos[1])
    recommand_columns = ['Date','proposition'] + list(recommand_inter.iloc[0][2:])
    recommand_inter.columns = recommand_columns
    recommand_inter['Date'] = recommand_inter['Date'].fillna(method = 'ffill')
    recommand_inter=recommand_inter.transpose()
    recommand = recommand.join( recommand_inter , how  = 'outer' , lsuffix='l', rsuffix='r')

    
recommand = recommand.transpose()  

recommand.index = range(len(recommand.index))
recommand = recommand[~recommand.loc[:,'Date'].str.contains('Parti 1').fillna(False)]
recommand.index = range(len(recommand.index))
recommand = recommand[~recommand['proposition'].isnull()]
#recommand = recommand[recommand_columns[1:]]


month_to_int = {'janvier' : '01' , 'février':'02', 'févirer':'02', 'mars':'03' , 'avril':'04' , 'mai':'05' , 'juin':'06','juillet':'07',\
              'aout':'08','septembre':'09','octobre':'10','novembre':'11','décembre':'12'}

def good_format_month (x) :
    if '.' in x :
        split_x = x.split('.')
        return split_x[0].zfill(2) + '.' + split_x[1].zfill(2) + '.' + split_x[2][:4]
    else :
        return str(x.split(' ')[0]).zfill(2) + '.' +  month_to_int[str(x.split(' ')[1]) ] + '.' + x[-4:] 

recommand.loc[:,'Date'] = recommand.loc[:,'Date'].map(lambda x : good_format_month(x))

recommand = recommand.loc[:,recommand.columns.drop_duplicates() ]
parties = list(recommand.columns.drop_duplicates())
parties.remove('Date')
parties.remove('proposition')

recommand = recommand.loc[:,['Date','proposition'] + parties]
recommand

C:\python3\lib\site-packages\pandas\core\indexing.py:601: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,Date,proposition,AdI,DS,Lega,MCR,PBD,PCS,PDC,PES,...,PSL,PST,PST,PST,PST,PVL,Parti 1),Rep.,UDC,UDF
0,12.02.2017,Naturalisation facilitée des étrangers de la t...,NaN,NaN,-,non,oui,NaN,oui,oui,...,NaN,NaN,NaN,NaN,NaN,oui,NaN,NaN,non,NaN
1,12.02.2017,Fonds pour les routes nationales et pour le tr...,NaN,NaN,-,oui,oui,NaN,oui,non,...,NaN,NaN,NaN,NaN,NaN,oui,NaN,NaN,oui,NaN
2,12.02.2017,Réforme de l'imposition des entreprises III,NaN,NaN,-,oui,oui,NaN,oui,non,...,NaN,NaN,NaN,NaN,NaN,oui,NaN,NaN,oui,NaN
3,21.05.2017,Loi sur l'énergie,NaN,NaN,-,non,oui,NaN,oui,oui,...,NaN,NaN,NaN,NaN,NaN,oui,NaN,NaN,non,NaN
7,28.02.2016,Initiative sur la réforme de l'imposition du c...,NaN,NaN,NaN,NaN,non,NaN,oui,non,...,NaN,NaN,NaN,NaN,NaN,non,NaN,NaN,oui,NaN
8,28.02.2016,Initiative pour le renvoi effectif des étrange...,NaN,NaN,NaN,NaN,non,NaN,non,non,...,NaN,NaN,NaN,NaN,NaN,non,NaN,NaN,oui,NaN
9,28.02.2016,Initiative contre la spéculation sur les denré...,NaN,NaN,NaN,NaN,non,NaN,non,oui,...,NaN,NaN,NaN,NaN,NaN,non,NaN,NaN,non,NaN
10,28.02.2016,Réfection du tunnel routier du Gothard,NaN,NaN,NaN,NaN,oui,NaN,oui,non,...,NaN,NaN,NaN,NaN,NaN,non,NaN,NaN,oui,NaN
11,05.06.2016,Initiative populaire «En faveur du service pub...,NaN,NaN,NaN,NaN,non,NaN,non,non,...,NaN,NaN,NaN,NaN,NaN,non,NaN,NaN,non,NaN
12,05.06.2016,Initiative populaire «Pour un revenu de base i...,NaN,NaN,NaN,NaN,non,NaN,non,oui,...,NaN,NaN,NaN,NaN,NaN,non,NaN,NaN,non,NaN


In [4]:
recommand.columns.drop_duplicates()

Index(['Date', 'proposition', 'AdI', 'DS', 'Lega', 'MCR', 'PBD', 'PCS', 'PDC',
       'PES', 'PEV', 'PLR', 'PLR (PRD) 3)', 'PLR 3)', 'PLS', 'PLS 3)', 'POCH',
       'PRD', 'PS', 'PSL', 'PST', 'PVL', 'Parti 1)', 'Rep.', 'UDC', 'UDF'],
      dtype='object')

In [5]:
def translate_choice (x) :
    if (x == 1 or str(x) == 'oui') :
        return 1
    elif ( x == 2 or str(x) == 'non'):
        return 0
    else :
        return -1

good_recommand = recommand.copy() 
good_recommand.loc[:,parties] = good_recommand.loc[:,parties].applymap(lambda x: translate_choice (x) )
good_recommand

,Date,proposition,AdI,DS,Lega,MCR,PBD,PCS,PDC,PES,...,PSL,PST,PST,PST,PST,PVL,Parti 1),Rep.,UDC,UDF
0,12.02.2017,Naturalisation facilitée des étrangers de la t...,-1,-1,-1,0,1,-1,1,1,...,-1,-1,-1,-1,-1,1,-1,-1,0,-1
1,12.02.2017,Fonds pour les routes nationales et pour le tr...,-1,-1,-1,1,1,-1,1,0,...,-1,-1,-1,-1,-1,1,-1,-1,1,-1
2,12.02.2017,Réforme de l'imposition des entreprises III,-1,-1,-1,1,1,-1,1,0,...,-1,-1,-1,-1,-1,1,-1,-1,1,-1
3,21.05.2017,Loi sur l'énergie,-1,-1,-1,0,1,-1,1,1,...,-1,-1,-1,-1,-1,1,-1,-1,0,-1
7,28.02.2016,Initiative sur la réforme de l'imposition du c...,-1,-1,-1,-1,0,-1,1,0,...,-1,-1,-1,-1,-1,0,-1,-1,1,-1
8,28.02.2016,Initiative pour le renvoi effectif des étrange...,-1,-1,-1,-1,0,-1,0,0,...,-1,-1,-1,-1,-1,0,-1,-1,1,-1
9,28.02.2016,Initiative contre la spéculation sur les denré...,-1,-1,-1,-1,0,-1,0,1,...,-1,-1,-1,-1,-1,0,-1,-1,0,-1
10,28.02.2016,Réfection du tunnel routier du Gothard,-1,-1,-1,-1,1,-1,1,0,...,-1,-1,-1,-1,-1,0,-1,-1,1,-1
11,05.06.2016,Initiative populaire «En faveur du service pub...,-1,-1,-1,-1,0,-1,0,0,...,-1,-1,-1,-1,-1,0,-1,-1,0,-1
12,05.06.2016,Initiative populaire «Pour un revenu de base i...,-1,-1,-1,-1,0,-1,0,1,...,-1,-1,-1,-1,-1,0,-1,-1,0,-1


In [6]:
recommand_with_yes = df[['Commune' , 'Votation' , 'Oui en %' ]]

def link_recomendation_Votation (x) :
    votation = ' '.join(x.split(' ')[1:])
    #print(votation)
    return good_recommand[good_recommand['proposition'].str.contains(votation)]

#recommand_with_yes[recommand_columns[2:]] = recommand_with_yes['Votation'].map()
 #                                                              good_recommand[recommand_columns[2:]]
recommand_with_yes
#[link_recomendation_Votation(x) for x in recommand_with_yes[recommand_with_yes['Commune'] == 'Aeugst am Albis']['Votation'].values]

pd.concat([recommand_with_yes[recommand_with_yes['Commune'] == 'Aeugst am Albis'], \
           pd.concat(recommand_with_yes[recommand_with_yes['Commune'] == 'Aeugst am Albis']['Votation'].map(\
                                            lambda x :link_recomendation_Votation(x) ) ) ] , axis = 1)


C:\python3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "Series"

In [16]:
count = 0
count1 = 0
count2 = 0

for date1, group1 in df[df['Commune'] == 'Aeugst am Albis'].groupby('Date'):
    count +=1
    for date2, group2 in good_recommand.groupby('Date'):
        count1+=1
        if (date1 == date2) :
            count2+=1
            print(date1,date2)
            
            
  

01.04.1990 01.04.1990
01.12.1985 01.12.1985
01.12.1996 01.12.1996
02.06.1991 02.06.1991
02.06.2002 02.06.2002
02.12.1984 02.12.1984
02.12.2001 02.12.2001
03.03.1991 03.03.1991
03.03.2002 03.03.2002
03.03.2013 03.03.2013
04.03.2001 04.03.2001
04.06.1989 04.06.1989
04.12.1983 04.12.1983
04.12.1988 04.12.1988
04.12.1994 04.12.1994
05.06.2005 05.06.2005
05.06.2016 05.06.2016
06.06.1982 06.06.1982
06.06.1993 06.06.1993
06.12.1992 06.12.1992
07.02.1999 07.02.1999
07.03.1993 07.03.1993
07.03.2010 07.03.2010
07.06.1998 07.06.1998
07.12.1986 07.12.1986
08.02.2004 08.02.2004
08.02.2009 08.02.2009
08.03.2015 08.03.2015
08.06.1997 08.06.1997
09.02.2003 09.02.2003
09.02.2014 09.02.2014
09.06.1985 09.06.1985
09.06.1996 09.06.1996
09.06.2013 09.06.2013
10.03.1985 10.03.1985
10.03.1996 10.03.1996
10.06.2001 10.06.2001
11.03.2007 11.03.2007
11.03.2012 11.03.2012
12.02.2017 12.02.2017
12.03.1995 12.03.1995
12.03.2000 12.03.2000
12.06.1988 12.06.1988
12.06.1994 12.06.1994
13.02.2011 13.02.2011
13.06.1999

In [ ]:
(recommand_with_yes[recommand_with_yes['Commune'] == 'Aeugst am Albis']['Votation'].map(\
                                            lambda x :link_recomendation_Votation(x) ).values)

In [ ]:
[recommand_with_yes